## Library 호출 및 path 설정

## 예외 처리

In [1]:
import pandas as pd
import numpy as np
import json
import requests
import unicodedata
from copy import deepcopy

path = "https://raw.githubusercontent.com/Algorithm-Coding-Test-Data-Analysis/algoview/main/dataAnalysis/macro/data.json"                
resp = requests.get(path)

In [2]:
def unicode_err(df):  # 회사명, 문제유형, 문제이름 중복 오류
    
    '''
    unicodedata.normalize() 함수는 문자열에 있는 유니코드 문자를 
    NFKC형식(문자열을 표준 호환성(KC) 형식으로 정규화)으로 정규화합니다.
    problem_name, problem_type, company_name 컬럼의 값 중 같은 값을 
    파이썬에서 다르게 인식하는 문제를 해결합니다.
    
    Args:
        data : (DataFrame | Series)

    Returns:
        data : (DataFrame | Series)
    '''
    
    df["company_name"] = df['company_name'].apply(lambda x: unicodedata.normalize('NFKC', x))
    df["problem_type"] = df['problem_type'].apply(lambda x: unicodedata.normalize('NFKC', x))
    return df

def prob_type_to_etc(value):  # 문제 유형이 아닌 카테고리를 "기타"로 분류
    
    '''
    오류로 인해 실제 문제 유형이 아닌 값들을 예외 처리
    추후 추가 혹은 삭제 될 수 있습니다.
    
    Args:
        data : (DataFrame | Series)

    Returns:
        data : (DataFrame | Series)
    '''
    
    err_lst = ['하은.js', '문자열과', '별찍기', '개수', '다이나믹', '준혁.js', 
               '에라토스테네스의체', '수정.js' ,'지현.js', '출력하기', '회전시키기', 
               '판별하기', '반복', '반복해서', '바꿔서', '돌리기', '숫자의', '구하기']
    if value in err_lst:
        return "기타"
    else:
        return value
    
    
def file_name_exception(json_file):
    err_lst = ["Lv1_프로그래머스_행렬의덧셈_구현_지현.js",  # 1차 cycle 예외 처리 (파일명 오류, 개선 될 예정)
               "Lv0_0000_뒤에서 5등까지_구현_수정.js",
               "Lv0_0000_특정 문자열로 끝나는 가장 긴 부분 문자열 찾기_구현_수정.js",
               "Lv0_0000_자릿수더하기_구현_지현.js",
               "Lv1_0000_프로그래머스_수박수박수박수박수박수__구현_재호.py",
               "Lv1_0000_프로그래머스_3진법 뒤집기_구현_재영.py"
              ]

    print("예외 처리 전 json 파일 길이: ", len(json_file))
    for error_file in err_lst:  # 1차 cycle 예외 처리 (파일명 오류, 개선 될 예정)
        del json_file[error_file]
    print("예외 처리 후 json 파일 길이: ", len(json_file))
    
    return json_file


## DataFrame으로 변환

In [3]:
def split_data_into_py_js(data, lang = "py"):  # Python, JavaScript로 data 분리
    
    '''
    전체 데이터인 DataFrame을 입력 받아
    Pytho과 JavaScript 별로 데이터를 분리

    Args:
        data (DataFrame): DataFrame으로 변환된 data
        language  (str): 언어 구분

    Returns:
        언어별 데이터 (DataFrame)
    '''
    
    print("-"*5, "split into python DataFrame & js DataFrame","-"*5)
    
    df = data.copy()
    if lang == "py":
        py_df = df[df["file_name"].str.contains(".py")].reset_index(drop = True)
        print("python DataFrame 길이: {0}".format(len(py_df)))
        return py_df
    
    elif lang == "js":
        js_df = df[df["file_name"].str.contains(".js")].reset_index(drop = True)
        print("js DataFrame 길이: {0}".format(len(js_df)))
        return js_df

##-----------------------------------------------------------------------------------------------------------------
    
def json_to_df(json_file):  # json_normalize 사용 불가, 따라서 methodcount 와 method명을 분리하기 위함
    
    '''
    json 형태인 데이터를 입력 받아
    DataFrame으로 변환
    ‘function_method’ column 안에 method 와 module이 함께 집계됩니다.

    Args:
        data : (json)

    Returns:
        data : (DataFrame)
    '''
    
    data_json = deepcopy(json_file)
    df = pd.DataFrame() # empty DataFrame
    for file_name in data_json.keys():
        tmp_df = pd.DataFrame(data_json[file_name])
        tmp_df = tmp_df.reset_index()
        df = pd.concat([df, tmp_df], axis = 0) 

        tmp_df_len = len(tmp_df)  # countmethod가 0인 파일 사전 처리 후 concat
        if tmp_df_len == 0:
            tmp_df = pd.DataFrame(data_json[file_name], index = [0]).reset_index()
            df = pd.concat([df, tmp_df], axis = 0)
            
    df = df.reset_index(drop = True).rename(columns = {"index" : "function_method"})  # method명 column으로 변환
    df = df[['file_name', 'level', 'year', 'company_name','problem_name', 'problem_type',  # column 순서 재배치
             'function_method', 'countmethod', 'module', 'line_count', 'code']]

    df = df.fillna(np.nan)
    df = unicode_err(df) # 회사명, 문제유형, 문제이름 중복 오류
    df["problem_type"] = df["problem_type"].apply(lambda x: prob_type_to_etc(x))  # 문제 유형이 아닌 카테고리를 "기타"로 분류
    df["company_name"] = df["company_name"].apply(lambda x: prob_type_to_etc(x))  # 문제 유형이 아닌 카테고리를 "기타"로 분류
    
    return df

##-----------------------------------------------------------------------------------------------------------------


def json_to_df_split_module_method(json_file):  # module, method 별도의 column으로 분리
        
    '''
    json 형태인 데이터를 입력 받아
    DataFrame으로 변환
    module 과 method를 별도의 column으로 분리
    
    Args:
        data : (json)

    Returns:
        data : (DataFrame)
    '''
    
    data = deepcopy(json_file)
    data_2 = deepcopy(json_file)
    
    for key in data.keys():
        del data[key]["module"]
    for key in data_2.keys():
        del data_2[key]["countmethod"]

    method_df = pd.DataFrame()
    for key in data.keys():
        if data[key]["countmethod"] == 0:
            data[key]["countmethod"] = {}
        tmp_df = pd.DataFrame(data[key]).reset_index()
        tmp_df = tmp_df.rename(columns = {"index":"method"})
        method_df = pd.concat([method_df, tmp_df], axis = 0) 
    del tmp_df

    module_df = pd.DataFrame()
    for key in data_2.keys():
        tmp_df = pd.DataFrame(data_2[key]).reset_index()
        tmp_df = tmp_df.rename(columns = {"index":"module_name"})
        module_df = pd.concat([module_df, tmp_df], axis = 0)
        module_df = module_df.drop(module_df.columns.to_list()[2:], axis = 1)
    
    
    df = method_df.merge(module_df, on = "file_name", how = "left")
    df = df[['file_name', 'level', 'year', 'company_name', 'problem_name',  # column 순서 재배치
             'problem_type', 'method',  'countmethod', 'module_name', 'line_count', 'code']]
    
    df = df.fillna(np.nan)
    df = unicode_err(df)  # 회사명, 문제유형, 문제이름 중복 오류
    df["problem_type"] = df["problem_type"].apply(lambda x: prob_type_to_etc(x))  # 문제 유형이 아닌 카테고리를 "기타"로 분류
    df["company_name"] = df["company_name"].apply(lambda x: prob_type_to_etc(x))  # 문제 유형이 아닌 카테고리를 "기타"로 분류

    return df


##-----------------------------------------------------------------------------------------------------------------

def json_to_df_dict():

    df = pd.read_json(path)
    df = df.T.reset_index(drop = True)
    df = df.fillna(np.nan)
    df = unicode_err(df)
    df["problem_type"] = df["problem_type"].apply(lambda x: prob_type_to_etc(x))  # 문제 유형이 아닌 카테고리를 "기타"로 분류
    df["company_name"] = df["company_name"].apply(lambda x: prob_type_to_etc(x))  # 문제 유형이 아닌 카테고리를 "기타"로 분류
    
    return df 


In [4]:
## 예외처리 코드
# data_json2 = deepcopy(dj)
# df = pd.DataFrame() # empty DataFrame
# i = 0
# lst = []
# for file_name in data_json2.keys():
#     i += 1
#     tmp_df = pd.DataFrame(data_json2[file_name])
#     tmp_df = tmp_df.reset_index()
    
#     tmp_df_len = len(tmp_df)
#     df_len = len(df)
    
#     df = pd.concat([df, tmp_df], axis = 0)
#     if tmp_df_len == 0:
#         tmp_df = pd.DataFrame(data_json2[file_name], index = [0]).reset_indext()
#         df = pd.concat([df, tmp_df], axis = 0)


        
#     print("len(df): {0}, tmp_df_len: {1} + df_len: {2}: {3}, i:{4}".format(len(df), tmp_df_len, df_len,  len(df) ==(tmp_df_len + df_len), i))


### 하나의 column 안에 method 와 module 

In [5]:
data_json = json.loads(resp.text)
data_json = file_name_exception(data_json)
data_df = json_to_df(data_json)
py_df = split_data_into_py_js(data_df, "py")
js_df = split_data_into_py_js(data_df, "js")

예외 처리 전 json 파일 길이:  604
예외 처리 후 json 파일 길이:  598
----- split into python DataFrame & js DataFrame -----
python DataFrame 길이: 557
----- split into python DataFrame & js DataFrame -----
js DataFrame 길이: 832


In [6]:
py_df.head()

,file_name,level,year,company_name,problem_name,problem_type,function_method,countmethod,module,line_count,code
0,Lv0_0000_프로그래머스_369 게임_구현_선빈.py,level0,0000,프로그래머스,369 게임,구현,str,1.0,NaN,3,def solution(order):\n order = str(order)\n...
1,Lv0_0000_프로그래머스_369 게임_구현_선빈.py,level0,0000,프로그래머스,369 게임,구현,.count,3.0,NaN,3,def solution(order):\n order = str(order)\n...
2,Lv0_0000_프로그래머스_369게임_구현_재호.py,level0,0000,프로그래머스,369게임,구현,str,3.0,NaN,6,def solution(order):\n result = 0\n resu...
3,Lv0_0000_프로그래머스_369게임_구현_재호.py,level0,0000,프로그래머스,369게임,구현,.count,3.0,NaN,6,def solution(order):\n result = 0\n resu...
4,Lv0_0000_프로그래머스_9로나눈나머지_구현_윤정.py,level0,0000,프로그래머스,9로나눈나머지,구현,list,1.0,NaN,7,def solution(number):\n arr = list(number)\...


In [7]:
py_df[py_df["file_name"] == "Lv1_2018_프로그래머스_소수 만들기_구현_재영.py"]

,file_name,level,year,company_name,problem_name,problem_type,function_method,countmethod,module,line_count,code
431,Lv1_2018_프로그래머스_소수 만들기_구현_재영.py,level1,2018,프로그래머스,소수 만들기,구현,sum,1.0,NaN,12,from itertools import combinations\ndef soluti...
432,Lv1_2018_프로그래머스_소수 만들기_구현_재영.py,level1,2018,프로그래머스,소수 만들기,구현,range,1.0,NaN,12,from itertools import combinations\ndef soluti...
433,Lv1_2018_프로그래머스_소수 만들기_구현_재영.py,level1,2018,프로그래머스,소수 만들기,구현,itertools,NaN,1.0,12,from itertools import combinations\ndef soluti...
434,Lv1_2018_프로그래머스_소수 만들기_구현_재영.py,level1,2018,프로그래머스,소수 만들기,구현,combinations,NaN,1.0,12,from itertools import combinations\ndef soluti...


---

### method, module columns 분리

In [8]:
data_json = json.loads(resp.text)
data_json = file_name_exception(data_json)
data_df = json_to_df_split_module_method(data_json)
py_df = split_data_into_py_js(data_df, "py")
js_df = split_data_into_py_js(data_df, "js")

예외 처리 전 json 파일 길이:  604
예외 처리 후 json 파일 길이:  598
----- split into python DataFrame & js DataFrame -----
python DataFrame 길이: 525
----- split into python DataFrame & js DataFrame -----
js DataFrame 길이: 760


In [9]:
py_df.head()

,file_name,level,year,company_name,problem_name,problem_type,method,countmethod,module_name,line_count,code
0,Lv0_0000_프로그래머스_369 게임_구현_선빈.py,level0,0000,프로그래머스,369 게임,구현,.count,3.0,NaN,3,def solution(order):\n order = str(order)\n...
1,Lv0_0000_프로그래머스_369 게임_구현_선빈.py,level0,0000,프로그래머스,369 게임,구현,str,1.0,NaN,3,def solution(order):\n order = str(order)\n...
2,Lv0_0000_프로그래머스_369게임_구현_재호.py,level0,0000,프로그래머스,369게임,구현,.count,3.0,NaN,6,def solution(order):\n result = 0\n resu...
3,Lv0_0000_프로그래머스_369게임_구현_재호.py,level0,0000,프로그래머스,369게임,구현,str,3.0,NaN,6,def solution(order):\n result = 0\n resu...
4,Lv0_0000_프로그래머스_9로나눈나머지_구현_윤정.py,level0,0000,프로그래머스,9로나눈나머지,구현,int,1.0,NaN,7,def solution(number):\n arr = list(number)\...


In [10]:
py_df[py_df["file_name"] == "Lv1_2018_프로그래머스_소수 만들기_구현_재영.py"]

,file_name,level,year,company_name,problem_name,problem_type,method,countmethod,module_name,line_count,code
380,Lv1_2018_프로그래머스_소수 만들기_구현_재영.py,level1,2018,프로그래머스,소수 만들기,구현,range,1.0,combinations,12,from itertools import combinations\ndef soluti...
381,Lv1_2018_프로그래머스_소수 만들기_구현_재영.py,level1,2018,프로그래머스,소수 만들기,구현,range,1.0,itertools,12,from itertools import combinations\ndef soluti...
382,Lv1_2018_프로그래머스_소수 만들기_구현_재영.py,level1,2018,프로그래머스,소수 만들기,구현,sum,1.0,combinations,12,from itertools import combinations\ndef soluti...
383,Lv1_2018_프로그래머스_소수 만들기_구현_재영.py,level1,2018,프로그래머스,소수 만들기,구현,sum,1.0,itertools,12,from itertools import combinations\ndef soluti...


---

### dict 형태의 column

In [11]:
data_df = json_to_df_dict()
py_df = split_data_into_py_js(data_df, "py")
js_df = split_data_into_py_js(data_df, "js")

----- split into python DataFrame & js DataFrame -----
python DataFrame 길이: 245
----- split into python DataFrame & js DataFrame -----
js DataFrame 길이: 359


In [12]:
py_df.head()

,file_name,level,year,company_name,problem_name,problem_type,countmethod,line_count,module,code
0,Lv0_0000_프로그래머스_369 게임_구현_선빈.py,level0,0000,프로그래머스,369 게임,구현,"{'str': 1, '.count': 3}",3,{},def solution(order):\n order = str(order)\n...
1,Lv0_0000_프로그래머스_369게임_구현_재호.py,level0,0000,프로그래머스,369게임,구현,"{'str': 3, '.count': 3}",6,{},def solution(order):\n result = 0\n resu...
2,Lv0_0000_프로그래머스_9로나눈나머지_구현_윤정.py,level0,0000,프로그래머스,9로나눈나머지,구현,"{'list': 1, 'range': 1, 'len': 1, 'int': 1}",7,{},def solution(number):\n arr = list(number)\...
3,Lv0_0000_프로그래머스_n의 배수 고르기_구현_선빈.py,level0,0000,프로그래머스,n의 배수 고르기,구현,0,3,{},"def solution(n, numlist):\n answer = [i for..."
4,Lv0_0000_프로그래머스_n의배수고르기_구현_재호.py,level0,0000,프로그래머스,n의배수고르기,구현,0,3,{},"def solution(n, numlist):\n result = [i for..."
